# ![](https://ga-dash.s3.amazonaws.com/production/assets/logo-9f88ae6c9c3871690e33280fcf557f33.png) Stats Review & Intro to SciPy
Week 2 | Lesson 5.1




### LEARNING OBJECTIVES
*After this lesson, you will be able to:*

- Explain Type I and Type II errors
- Explain t-testing and demonstrate it with scipy
- Contrast t-testing with simulation solutions

### LESSON GUIDE
| TIMING  | TYPE  | TOPIC  |
|:-:|---|---|
| 10 min  | [Demo](#introduction)   | Significance levels, Type I and Type II errors  |
| 10 min  | [Demo](#demo)  | Law of large numbers and central limit theorem  |
| 10 min  | [Demo / Guided Practice](#demo)  | T-testing revisited  |
| 20 min  | [Independent Practice](#ind-practice)  | T-testing  |
| 10 min  | [Demo /Guided Practice](#demo)  | Computational approaches  |
| 30 min  | [Independent Practice](#ind-practice)  |  Computational statistics |
| 5 min  | [Conclusion](#conclusion)  | |



## Significance levels, Type I and Type II errors

Type I errors occur when the researcher rejects a null hypothesis when it is actually true. The probability of committing a Type I error is called the significance level, often denoted $\alpha$.

A Type II error occurs when the researcher wrongly accepts a null hypothesis that is false.  The probability of committing a Type II error is often denoted by $\beta$.

$$\alpha\ =\ P(Reject\ H_0\ |\ H_0\ is\ true) = P(Type\ I\ error)$$


$$\beta\ =\ P(Not Reject\ H_0\ |\ H_a\ is\ true) = P(Type\ II\ error)$$

$$\beta\ =\ P(Not Reject\ H_0\ |\ H_0\ is\ false) = P(Type\ II\ error)$$

## Law of large numbers

If $Y_1,\ Y_2\ ...,\ Y_n$ are independently and identically distributed (i.i.d) with mean $\mu$ and finite variance, a sample mean converges in probability to $\mu$.

This means that for sufficiently large N, $\bar{Y}\ -\ \mu$ is ~ 0.

## Central limit theorem

- If $Y_1,\ Y_2\ ...,\ Y_n$ are i.i.d with mean $\mu$ and variance $\sigma^2$, then when *n* is large, sample mean $\hat{\mu}$ is approximately normally distributed with mean $\mu$ and variance $\frac{\sigma^2}{n}$.
- $\hat{\mu}$ is asymptotically normally distributed
- So? Well, this allows us to assume that some random variables are normally distributed, and to make inferences about the likelihood of observations drawn from that distribution. For instance, it implies:

$$\frac{\hat{\mu}\ -\ \mu}{\sigma/\sqrt{n}}\ \sim \ N(0,1)$$

## T-tests revisited

Does $\frac{\hat{\mu}\ -\ \mu}{\sigma/\sqrt{n}}$ look familiar? If we use the sample standard deviation in the denominator, that's the t-statistic!

$$\frac{\hat{\mu}\ -\ \mu}{s/\sqrt{n}}$$

And if n is large, then the value of the t-statistic is approximately normally distributed. (If n is small and the sample observations are normally distributed, then it has a t-distribution.)

$\frac{s}{\sqrt{n}}$ is also called the standard error, $s.e.of  \hat\mu $

* Numerator: signal
* Denominator: noise
* Way of standardizing mean (by putting mean at 0)
    * Assume a probability distribution of N(0,1)

## P-values and hypothesis testing, revisited

If we can assume a probability distribution, e.g. N(0,1), then we can calculate the likelihood of seeing some value given that assumption.

![](./norm_dist_probs.jpg)

### Hypothesis testing with t-tests, review

Let's say 1165 bootcamp applicants take a GA admissions test in 2017, with an average score of 60.86 and a standard deviation of 8.02. The expected score for all bootcamp applicants (not just GA) is 59. Do GA applicants have the same expected score?

We can estimate $\mu$, the GA applicant population mean, with our sample mean. $\hat{\mu} = \bar{Y} = 60.86$. The sample standard deviation *s* is 8.02.

Standard error of the estimate is then $se(\hat{\mu}) = \frac{s}{\sqrt{n}} = \frac{8.02}{\sqrt{1165}} = 0.235$

> What are our null and alternative hypotheses?

* Null hypothesis is that there is no difference
* Alternative hypothesis is that there is a difference (not 0)

$$H_0: \hat\mu = 59$$
$$H_a: \hat\mu \neq 59$$

Under $H_0,\ t = \frac{\hat{\mu} - 59}{se(\hat{\mu})}$ is approximate normally distributed with N(0,1). If t falls far on the tail, the p-value is low and we'll reject $H_0$.

Calculate the t-statistic and [look up its p-value](https://graphpad.com/quickcalcs/PValue1.cfm]).



$$t = \frac{60.86 - 59}{0.2350} = 7.915$$

P < 0.0001

## Again, but now with scipy ('skippy'?)

In [1]:
from scipy import stats
import numpy as np
np.random.seed(7654567)  # fix seed to get the same result - subject of our lab
rvs = stats.norm.rvs(loc=60.86, scale=8.02, size=(1165))
print rvs
#loc = mean; scale = sd

# Note that the mean and std of our generated data aren't precisely the same.
print np.std(rvs), np.mean(rvs)


[ 53.26524642  58.64445885  59.26615073 ...,  57.78177772  55.42126582
  65.92945246]
8.05037070851 61.1582143415


In [2]:
stats.ttest_1samp(rvs, 59)

Ttest_1sampResult(statistic=9.1465051826423593, pvalue=2.5558469858139369e-19)

## Which scipy function to use?

A few common t-tests include:
- One-sample t-test. Used to determine whether a hypothesized population
    mean differs significantly from an observed sample mean.
- Two-sample t-test. Used to determine whether the difference between samples means differs significantly from the              hypothesized difference between population means.
- Paired t-test. Used to test the significance of the difference
    between paired means.

Scipy has methods for all of these, and more. Which one do we want?



From the documentation, we want `scipy.stats.ttest_1samp`, for our one-sample t-test.

To verify, compare our formula with the scipy code, [here](https://github.com/scipy/scipy/blob/v0.14.0/scipy/stats/stats.py#L3037)


```python
 a, axis = _chk_asarray(a, axis)
    n = a.shape[axis]
    df = n - 1

    d = np.mean(a, axis) - popmean
    v = np.var(a, axis, ddof=1)
    denom = np.sqrt(v / float(n))

    t = np.divide(d, denom)
    t, prob = _ttest_finish(df, t)

    return t, prob
    ```

## Look good? Okay.

In [3]:
# Test if mean of random sample is equal to true mean, and different mean.
# We reject the null hypothesis in the second case and don’t reject it in the first case.

print stats.ttest_1samp(rvs,59.0) #reject null hypothesis, p that given hypothesized population mean (59), what
#is probability of seeing rvs
print stats.ttest_1samp(rvs,61.1582143415) #not reject null hypothesis

Ttest_1sampResult(statistic=9.1465051826423593, pvalue=2.5558469858139369e-19)
Ttest_1sampResult(statistic=-9.0338332344805128e-12, pvalue=0.99999999999279354)


<a name="ind-practice"></a>
## Independent Practice: classic t-tests (20 minutes)

In pairs or trios, look at the SAT test data from Project 1. (We'll assume it's a sample of results, rather than the population results.) Together, form null and alternative hypotheses about some of the scores. 
(E.g., H0: the mean difference between states' verbal and math scores is 0; or H0: the population math score is 550.)

Choose a significance level and conduct an appropriate t-test.

- [t-tests](http://iaingallagher.tumblr.com/post/50980987285/t-tests-in-python)
- [t distribution](http://stattrek.com/probability-distributions/t-distribution.aspx)

In [4]:
import pandas as pd
sat = pd.read_csv('/Users/kristensu/Dropbox/GA-DSI/DSI-copy/projects/projects-weekly/project-01/assets/sat_scores.csv')
sat.head()

,State,Rate,Verbal,Math
0,CT,82,509,510
1,NJ,81,499,513
2,MA,79,511,515
3,NY,77,495,505
4,NH,72,520,516


In [5]:
verbal_mean = np.mean(sat['Verbal'])
print verbal_mean
stats.ttest_1samp(sat['Verbal'], 800)

532.019230769


Ttest_1sampResult(statistic=-58.142487084417603, pvalue=2.7364912639127121e-48)

In [6]:
math_mean = np.mean(sat['Math'])
print math_mean
stats.ttest_1samp(sat['Math'],550)

531.5


Ttest_1sampResult(statistic=-3.7041646377908148, pvalue=0.00052211980150622395)

In [7]:
stats.ttest_1samp(sat['Math'],np.mean(sat['Verbal']))

Ttest_1sampResult(statistic=-0.10396304076958617, pvalue=0.91760644500200694)

In [8]:
#just testing out; stats.ttest_ind assumes identical variances (works in this instance)
stats.ttest_ind(sat['Math'],sat['Verbal'])

Ttest_indResult(statistic=-0.076401246238828047, pvalue=0.93924953595020755)

<a name="t-testing"></a>
## Demo/Guided Practice: computational approaches (10 minutes)

Now that computational power is cheap and available (and you know Python!), we have an alternative way of approaching these questions: iteratively calculate the probability of observing some result.

For example:

```Python
# Simulating a binomial variable (e.g. seeing heads in 20 out of 30 coin flips )
m = 0
for i in range(10000):
    trials = np.random.randint(2, size = 30)
    if (trials.sum() >= 20):
        m += 1
p = m / 10000.0
p
```

> Check: what is this doing?

In [30]:
m = 0
for i in range(100000):
    trials = np.random.randint(2, size = 30)
    if (trials.sum() >= 20):
        m += 1
p = m / 100000.0
p

0.0494

In [26]:
trials = ((np.random.randn(1,30))**2)
m = 0
for i in range(1000):
    if trials.sum
    

SyntaxError: invalid syntax (<ipython-input-26-84148a58eca3>, line 4)

In [22]:
m = 0
for i in range(100000):
    trials = np.random.randint(2)
    if (trials.sum() >= 5):
        m += 1
p = m / 100000.0
p

TypeError: randn() got an unexpected keyword argument 'size'

This was an example if **simulating** your experiment -- you can do this if you have an a priori model of what happens.

If you don't have an a priori model, another option is **shuffling** results:

(Example from: http://cs.nyu.edu/shasha/papers/StatisticsIsEasyExcerpt.html)

"Imagine we have given some people a placebo and others a drug. The measured improvement (the more positive the better) is

Placebo: 54 51 58 44 55 52 42 47 58 46

Drug: 54 73 53 70 73 68 52 65 65




"As you can see, the drug seems more effective on the average (the average measured improvement is 63.7 for the drug and 50.7 for the placebo). But is this difference in the average real? Formula-based statistics would use a t-test which entails certain assumptions about normality and variance, but we are going to look just at the samples themselves and shuffle the labels."



In [36]:
placebo = pd.DataFrame([54, 51, 58, 44, 55, 52, 42, 47, 58, 46])
drug = pd.DataFrame([54, 73, 53, 70, 73, 68, 52, 65, 65])
place_mean = np.mean(placebo)
drug_mean = np.mean(drug)

print place_mean
print drug_mean

0    50.7
dtype: float64
0    63.666667
dtype: float64


In [44]:
stats.ttest_1samp(placebo, drug_mean)

Ttest_1sampResult(statistic=0   -7.173021
dtype: float64, pvalue=array([  5.23390388e-05]))

In [40]:
combined = pd.concat([placebo, drug], join='outer', ignore_index=True)
print combined

     0
0   54
1   51
2   58
3   44
4   55
5   52
6   42
7   47
8   58
9   46
10  54
11  73
12  53
13  70
14  73
15  68
16  52
17  65
18  65


What this means can be illustrated as follows. We put all the people in a table having two columns value and label (P for placebo and D for drug).

| value | label |
|:-:|---|
|54	| P |
|51	| P |
|58	| P |
|44	| P |
|55	| P |
|52	| P |
|42	| P |
|47	| P |
|58	| P |
|46	| P |
|54	| D |
|73	| D |
|53	| D |
|70	| D |
|73	| D |
|68	| D |
|52	| D |
|65	| D |
|65	| D |



Shuffling the labels means that we will take the Ps and Ds and randomly distribute them among the patients. (Technically, we do a uniform random permutation of the label column.)

This might give:

| value | label |
|:-:|---|
|54	| P 
|51	| P
|58	| D
|44	| P
|55	| P
|52	| D
|42	| D
|47	| D
|58	| D
|46	| D
|54	| P
|73	| P
|53	| P
|70	| D
|73	| P
|68	| P
|52	| D
|65	| P
|65	| D



We can then look at the difference in the average P value vs. the average D value here. We get an average of 59.0 for P and 54.4 for D. We repeat this shuffle-then-measure procedure 10,000 times and ask what fraction of time we get a difference between drug and placebo greater than or equal to the measured difference of 63.7 - 50.7 = 13. The answer in this case is under 0.001.""

> Check: what are the benefits of a computational strategy? The risks?

<a name="ind-practice"></a>
## Independent Practice: finding probabilities computationally (30 minutes)

In pairs or trios, design and code a computational way of finding the probability of rolling a 6 at least one-third of the time on a fair die.

In [63]:
die = np.random.randint(6, size=1)
six = 0

In [64]:
die

array([3])

In [126]:
six = 0
count = 0

for i in range(10000):
    die_roll = np.random.randint(1, 7, size=100)
    for x in die_roll:
        if x == 6:
            six += 1
        if six/100 == 1/3:
            count += 1.0

count/10000

0.0615

In [108]:
die_mean = np.mean([1,2,3,4,5,6])
die_std = np.std([1,2,3,4,5,6])

In [112]:
die_roll_2 = stats.norm.rvs(loc=die_mean, scale=die_std, size=(10000))

In [113]:
stats.ttest_1samp(die_roll_2, (1/3))

Ttest_1sampResult(statistic=205.53252681579355, pvalue=0.0)

In [ ]:
#Now try finding the probability of seeing a difference of at least 0.5192 between the 
#mean verbal and math scores in your SAT dataset, assuming there is, in truth, no difference between these means

<a name="conclusion"></a>
## Conclusion (5 mins)

- We make trade-offs between risking Type I and Type II errors
- There are varieties of t-tests, and scipy methods for conducting them
- Simulations / computation strategies are an alternative to parametric statistical inference